In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

### Problem Setting

In [2]:
# init
steepness = 50    # steepness factor
num_blocks = 100  # number of expression blocks
num_data = 9100   # number of data
test_size = 100   # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [3]:
# parameters as input data
b_low, b_high = 1.0, 8.0
a_low, a_high = 0.5, 4.5
b_train = np.random.uniform(b_low, b_high, (train_size, 1)).astype(np.float32)
b_test  = np.random.uniform(b_low, b_high, (test_size, 1)).astype(np.float32)
b_dev   = np.random.uniform(b_low, b_high, (val_size, 1)).astype(np.float32)
a_train = np.random.uniform(a_low, a_high, (train_size, num_blocks)).astype(np.float32)
a_test  = np.random.uniform(a_low, a_high, (test_size, num_blocks)).astype(np.float32)
a_dev   = np.random.uniform(a_low, a_high, (val_size, num_blocks)).astype(np.float32)

In [4]:
# nm datasets
from neuromancer.dataset import DictDataset
data_train = DictDataset({"b":b_train, "a":a_train}, name="train")
data_test = DictDataset({"b":b_test, "a":a_test}, name="test")
data_dev = DictDataset({"b":b_dev, "a":a_dev}, name="dev")
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=True)

### Exact Solver

In [5]:
from src.problem import msRosenbrock
model = msRosenbrock(steepness, num_blocks, timelimit=60)

### Rounding Classification

In [6]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [7]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 64            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [8]:
# set problem
import neuromancer as nm
from src.func.layer import netFC
from src.func import roundGumbelModel
from src.problem import nmRosenbrock, rosenbrockEquality
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks-3, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b", "a"], ["z"], name="smap")
# linear constraint encode
encoding = rosenbrockEquality(num_blocks, input_key="z", output_key="x")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks-3)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["b", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, equality_encoding=encoding, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, encoding, rnd]).to("cuda")
loss_fn = nmRosenbrock(["b", "a", "x_rnd"], steepness, num_blocks, penalty_weight)

In [9]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs=epochs, patience=patience, warmup=warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Iters 0, Validation Loss: 25769.48
Epoch 0, Iters 125, Training Loss: 12094.68, Validation Loss: 4129.36
Epoch 1, Iters 250, Training Loss: 2738.61, Validation Loss: 1599.29
Epoch 2, Iters 375, Training Loss: 2032.03, Validation Loss: 1433.68
Epoch 3, Iters 500, Training Loss: 1822.36, Validation Loss: 1152.26
Epoch 4, Iters 625, Training Loss: 1507.45, Validation Loss: 1156.96
Epoch 5, Iters 750, Training Loss: 1431.31, Validation Loss: 1204.62
Epoch 6, Iters 875, Training Loss: 1327.99, Validation Loss: 975.45
Epoch 7, Iters 1000, Training Loss: 1283.07, Validation Loss: 862.90
Epoch 8, Iters 1125, Training Loss: 1128.29, Validation Loss: 931.25
Epoch 9, Iters 1250, Training Loss: 1135.00, Validation Loss: 886.01
Epoch 10, Iters 1375, Training Loss: 1136.96, Validation Loss: 826.25
Epoch 11, Iters 1500, Training Loss: 1058.53, Validation Loss: 777.54
Epoch 12, Iters 1625, Training Loss: 956.90, Validation Loss: 766.02
Epoch 13, Iters 1750, Training Loss: 957.97, Validation L

In [10]:
params, sols, objvals, mean_viols, max_viols, num_viols, elapseds = [], [], [], [], [], [], []
for b, a in tqdm(list(zip(b_test, a_test))):
    # data point as tensor
    datapoints = {"b": torch.tensor(np.array([b]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    viol = model.cal_violation()
    mean_viols.append(np.mean(viol))
    max_viols.append(np.max(viol))
    num_viols.append(np.sum(viol > 1e-6))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param": params,
                    "Sol": sols,
                    "Obj Val": objvals,
                    "Mean Violation": mean_viols,
                    "Max Violation": max_viols,
                    "Num Violations": num_viols,
                    "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Num Violations"] > 0)))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 57.91it/s]


          Obj Val  Mean Violation  Max Violation  Num Violations  Elapsed Time
count  100.000000           100.0          100.0           100.0    100.000000
mean   497.850032             0.0            0.0             0.0      0.003545
std     86.855250             0.0            0.0             0.0      0.000991
min    312.995691             0.0            0.0             0.0      0.001997
25%    429.101735             0.0            0.0             0.0      0.003001
50%    488.077553             0.0            0.0             0.0      0.003294
75%    561.077152             0.0            0.0             0.0      0.003959
max    689.484268             0.0            0.0             0.0      0.006710
Number of infeasible solution: 0


### Learnable Threshold

In [11]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [12]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 64            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [13]:
# set problem
import neuromancer as nm
from src.func.layer import netFC
from src.func import roundThresholdModel
from src.problem import nmRosenbrock, rosenbrockEquality
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks-3, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b", "a"], ["z"], name="smap")
# linear constraint encode
encoding = rosenbrockEquality(num_blocks, input_key="z", output_key="x")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks-3)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["b", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                          int_ind=model.int_ind, continuous_update=True, equality_encoding=encoding, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, encoding, rnd]).to("cuda")
loss_fn = nmRosenbrock(["b", "a", "x_rnd"], steepness, num_blocks, penalty_weight)

In [14]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs=epochs, patience=patience, warmup=warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Iters 0, Validation Loss: 22756.81
Epoch 0, Iters 125, Training Loss: 10810.08, Validation Loss: 2614.14
Epoch 1, Iters 250, Training Loss: 1935.77, Validation Loss: 1301.65
Epoch 2, Iters 375, Training Loss: 1556.54, Validation Loss: 1053.93
Epoch 3, Iters 500, Training Loss: 1339.13, Validation Loss: 1026.92
Epoch 4, Iters 625, Training Loss: 1292.43, Validation Loss: 927.83
Epoch 5, Iters 750, Training Loss: 1224.03, Validation Loss: 1657.92
Epoch 6, Iters 875, Training Loss: 1224.14, Validation Loss: 1211.57
Epoch 7, Iters 1000, Training Loss: 999.39, Validation Loss: 837.11
Epoch 8, Iters 1125, Training Loss: 928.84, Validation Loss: 745.00
Epoch 9, Iters 1250, Training Loss: 873.65, Validation Loss: 697.63
Epoch 10, Iters 1375, Training Loss: 822.52, Validation Loss: 746.11
Epoch 11, Iters 1500, Training Loss: 801.90, Validation Loss: 702.92
Epoch 12, Iters 1625, Training Loss: 786.91, Validation Loss: 656.25
Epoch 13, Iters 1750, Training Loss: 765.94, Validation Loss: 

In [15]:
params, sols, objvals, mean_viols, max_viols, num_viols, elapseds = [], [], [], [], [], [], []
for b, a in tqdm(list(zip(b_test, a_test))):
    # data point as tensor
    datapoints = {"b": torch.tensor(np.array([b]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    viol = model.cal_violation()
    mean_viols.append(np.mean(viol))
    max_viols.append(np.max(viol))
    num_viols.append(np.sum(viol > 1e-6))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param": params,
                    "Sol": sols,
                    "Obj Val": objvals,
                    "Mean Violation": mean_viols,
                    "Max Violation": max_viols,
                    "Num Violations": num_viols,
                    "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Num Violations"] > 0)))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 53.35it/s]


          Obj Val  Mean Violation  Max Violation  Num Violations  Elapsed Time
count  100.000000           100.0          100.0           100.0    100.000000
mean   561.447941             0.0            0.0             0.0      0.003749
std    115.675314             0.0            0.0             0.0      0.001547
min    334.502777             0.0            0.0             0.0      0.002000
25%    498.454217             0.0            0.0             0.0      0.003000
50%    553.078944             0.0            0.0             0.0      0.003079
75%    616.772741             0.0            0.0             0.0      0.004106
max    971.378564             0.0            0.0             0.0      0.011631
Number of infeasible solution: 0
